## Sequential perturbations

02/16/2025

Alan M H Beem

In [1]:
N = 40
# K = 2
from package.abn_mir_helper_functions import *
from package.abn_mir_plotting_functions import select_network
from package.bn_graph_methods import *
from package.plotting import get_colors, binary_states
from random import SystemRandom as rand
from seq_pert_as_py import seq_pert_report

#### Problem specification:

Find a sequence of unit perturbations (p1, p2) such that for all cycles (except goal cycle) there exists an interval L such that a run-in from the perturbed state (i.e. p1 has been applied, L steps have occurred, and p2 has been applied) would terminate in cycle detection of the goal cycle.

---

Secondarily, let the Boolean network be a deterministic finite automaton that detects the goal cycle states

---
Further,

Let there be a 5% chance of unit perturbation;

Alternatively,

Let there be a % chance of unit perturbation that is also a function of total change in Boolean states per step;

#### Solution method:

#### Adjacency matrix from transitions matrix

After forming an adjacency matrix (with a 1 for each non-zero entry of the transitions matrix, and a 0 otherwise), by raising the matrix to successively higher powers, starting from 1, we could determine whether the goal states (cycle) are reachable from all other cycles, in some number of perturbations (the values of the non-zero entries after multiplication).

But for this:

Algorithm:

best sequences <- list

For each perturbable node, form a matrix _A<sub>p<sub>k</sub></sub>_, with elements _a<sub>i,j</sub>_, 1 if perturbation of node _k_ causes a network in state _i_ to transitions to state _j_, otherwise 0.

For all ordered pairs of matrices A<sub>p<sub>k</sub></sub>:

Does A<sub>p<sub>k<sub>1</sub></sub></sub> $*$ A<sub>p<sub>k<sub>2</sub></sub></sub> have all non-zero entries in the column of goal cycle index, for all rows except the row of goal cycle index?  | In retrospect, it would be good to include that row.

If so, there does exist such a sequence of perturbations, separated by an interval L: 0 ≤ L ≤ longest cycle length - 1, such that applying p1, waiting L steps, and applying p2 will result in a goal state.

Otherwise, which sequence has the most in the column for the goal cycle? or, the greatest sum in the column for the goal cycle?

### Set up a network for which to search for sequential perturbations effecting goal states

In [ ]:
# setup
net = select_network(
    num_nodes=N,
    minimum_max_cycle_length=15,
    maximum_max_cycle_length=35,
    minimum_number_of_cycles=10,
    maximum_number_of_cycles=30,
    maximum_number_of_networks=1024,
    iterations_limit=400,
)[
    0
]  # select net also returns all generated nets, used in bool_main for appendix figures
net.add_cycles(1000)
print(net)
net.compute_unit_perturbations_matrix(sort_selection=1, compute_over_t_u=False)
setup_colors = get_colors(
    len(net.bn_collapsed_cycles) + 10, True
)  # [[1, 1, 1, 1]]  # [[0.3, 0.3, 0.3, 0.5]]
avg_color = [
    sum(setup_colors[i][0] for i in range(len(setup_colors))) / len(setup_colors),
    sum(setup_colors[i][1] for i in range(len(setup_colors))) / len(setup_colors),
    sum(setup_colors[i][2] for i in range(len(setup_colors))) / len(setup_colors),
    sum(setup_colors[i][3] for i in range(len(setup_colors))) / len(setup_colors) / 10,
]
setup_colors.append(avg_color)

#### Cycle states as binary numbers, goal cycle states as binary values
Each state is taken to be a binary integer (ex: for N=2, a state 01<sub>2</sub> -> 1, a state 11<sub>2</sub> -> 3)

In [ ]:
# goal states as a given cycle
goal_cycle_index = rand().randrange(0, len(net.bn_collapsed_cycles.cycle_records))
fig = binary_states(net, setup_colors, goal_cycle_index, plt)
plt.show()

#### Applying the algorithm from solution method:

In [ ]:
matrices = [
    np.zeros_like(net.cycles_unit_perturbations_transition_matrix) for _ in net.nodes
]
all_matrix = np.zeros_like(net.cycles_unit_perturbations_transition_matrix)
for record in net.cycles_unit_perturbations_records:
    if record.end_index is not None:
        matrices[record.perturbed_node_index][record.start_index][
            record.end_index
        ] = 1  # 1 represents adjacency
        all_matrix[record.start_index][
            record.end_index
        ] = 1  # 1 represents adjacency  # this one is for a secondary question
max_good_rows = 0
found_sequences_2 = []
# working_sequences_2 = []  # initial output -> thoughts: requiring that the perturbation not make the goal state change is important, but may be overly restrictive
best_sequence = None
for i in range(len(matrices)):
    for j in range(len(matrices)):
        matrix_product = matrices[i] @ matrices[j]
        good_rows = 0
        for row in matrix_product:
            if row[goal_cycle_index] > 0:
                good_rows += 1
        if good_rows == len(net.bn_collapsed_cycles) - 1:
            found_sequences_2.append((i, j))
        if good_rows > max_good_rows:
            max_good_rows = good_rows
            best_sequence = (i, j)
print(
    f"{len(found_sequences_2)} sequences of perturbations unanimously effect goal state"
)
print(
    f"results:\nsequence: {best_sequence}\nnumber of cycles for which sequence -> goal: {max_good_rows}"
)

#### Initially, no, at least with the network I was looking at, there is not such a sequence of perturbations.

However, there are often some that are close, and with noise, they do pretty good.

#### What about with another perturbation?

In [ ]:
matrices = [
    np.zeros_like(net.cycles_unit_perturbations_transition_matrix) for _ in net.nodes
]
all_matrix = np.zeros_like(net.cycles_unit_perturbations_transition_matrix)
for record in net.cycles_unit_perturbations_records:
    if record.end_index is not None:
        matrices[record.perturbed_node_index][record.start_index][
            record.end_index
        ] = 1  # 1 represents adjacency
        all_matrix[record.start_index][
            record.end_index
        ] = 1  # 1 represents adjacency  # this one is for a secondary question
max_good_rows = 0
found_sequences_3 = []
best_sequence_3 = None
# working_sequences_3 = []
for i in range(len(matrices)):
    for j in range(len(matrices)):
        m_i_j = matrices[i] @ matrices[j]
        for k in range(len(matrices)):
            matrix_product = m_i_j @ matrices[k]
            good_rows = 0
            for row in matrix_product:
                if row[goal_cycle_index] > 0:
                    good_rows += 1
            if good_rows == len(net.bn_collapsed_cycles) - 1:
                found_sequences_3.append((i, j, k))
            if good_rows > max_good_rows:
                max_good_rows = good_rows
                found_sequence = (i, j, k)
print(
    f"{len(found_sequences_3)} sequences of perturbations unanimously effect goal state"
)
print(
    f"results:\nsequence: {found_sequence}\nnumber of cycles for which sequence -> goal: {max_good_rows}"
)

#### Progression of distribution of states from uniform initial conditions (1 for each cycle state), BN ≠ DFA

##### With two unit perturbations

In [ ]:
leg = plt.figure()
leg.set_size_inches(10, 2)
leg_ax = plt.axes((0, 0, 1, 1))
plt.text(0, 0, "test")
plt.Rectangle

leg.add_axes(leg_ax)
plt.show()

fig = seq_pert_report(
    p1=best_sequence[0],
    p2=best_sequence[1],
    goal_cycle_index=goal_cycle_index,
    net=net,
    cycle_colors=setup_colors,
    total_steps=500,
    progress_div=1,
    goal_bool=False,
)
fig.set_size_inches(10, 30)
plt.show()
# color by terminal state label

##### With three unit perturbations

In [ ]:
fig = seq_pert_report(
    p1=found_sequence[0],
    p2=found_sequence[1],
    goal_cycle_index=goal_cycle_index,
    net=net,
    cycle_colors=setup_colors,
    total_steps=500,
    progress_div=1,
    goal_bool=True,
    p3=found_sequence[2],
)
fig.set_size_inches(10, 30)
plt.show()
# color by terminal state label

#### With noise:

##### With two unit perturbations:

In [ ]:
fig = seq_pert_report(
    p1=best_sequence[0],
    p2=best_sequence[1],
    goal_cycle_index=goal_cycle_index,
    net=net,
    cycle_colors=setup_colors,
    total_steps=500,
    progress_div=1,
    goal_bool=True,
    with_noise=0.05,
)
fig.set_size_inches(10, 30)
plt.show()
# color by terminal state label

##### With three unit perturbations

In [ ]:
fig = seq_pert_report(
    p1=found_sequence[0],
    p2=found_sequence[1],
    goal_cycle_index=goal_cycle_index,
    net=net,
    cycle_colors=setup_colors,
    total_steps=500,
    progress_div=1,
    goal_bool=True,
    p3=found_sequence[2],
    with_noise=0.05,
)
fig.set_size_inches(10, 30)
plt.show()
# color by terminal state label

#### With noise "BV" noise (Boolean Velocity):

A more limited set of transitions

#### Progression of distribution of states from uniform initial conditions (1 for each cycle state)

##### With two unit perturbations

In [ ]:
fig = seq_pert_report(
    p1=best_sequence[0],
    p2=best_sequence[1],
    goal_cycle_index=goal_cycle_index,
    net=net,
    cycle_colors=setup_colors,
    total_steps=1000,
    progress_div=1,
    goal_bool=True,
)
fig.set_size_inches(10, 30)
plt.show()
# color by terminal state label

##### With three unit perturbations

In [ ]:
fig = seq_pert_report(
    p1=found_sequence[0],
    p2=found_sequence[1],
    goal_cycle_index=goal_cycle_index,
    net=net,
    cycle_colors=setup_colors,
    total_steps=500,
    progress_div=1,
    goal_bool=True,
    p3=found_sequence[2],
)
fig.set_size_inches(10, 30)
plt.show()
# color by terminal state label

#### With noise:

##### With two unit perturbations:

In [ ]:
fig = seq_pert_report(
    p1=best_sequence[0],
    p2=best_sequence[1],
    goal_cycle_index=goal_cycle_index,
    net=net,
    cycle_colors=setup_colors,
    total_steps=500,
    progress_div=1,
    goal_bool=True,
    with_noise=0.05,
)
fig.set_size_inches(10, 30)
plt.show()
# color by terminal state label

##### With three unit perturbations

In [ ]:
fig = seq_pert_report(
    p1=found_sequence[0],
    p2=found_sequence[1],
    goal_cycle_index=goal_cycle_index,
    net=net,
    cycle_colors=setup_colors,
    total_steps=500,
    progress_div=1,
    goal_bool=True,
    p3=found_sequence[2],
    with_noise=0.05,
)
fig.set_size_inches(10, 30)
plt.show()
# color by terminal state label

#### With noise "BV" noise (Boolean Velocity):

A more limited set of transitions

In [ ]:
fig = seq_pert_report(
    p1=best_sequence[0],
    p2=best_sequence[1],
    goal_cycle_index=goal_cycle_index,
    net=net,
    cycle_colors=setup_colors,
    total_steps=500,
    progress_div=1,
    goal_bool=True,
    with_noise="bv",
)
fig.set_size_inches(10, 30)
plt.show()
# color by terminal state label

In [ ]:
fig = seq_pert_report(
    p1=found_sequence[0],
    p2=found_sequence[1],
    goal_cycle_index=goal_cycle_index,
    net=net,
    cycle_colors=setup_colors,
    total_steps=500,
    progress_div=1,
    goal_bool=True,
    p3=found_sequence[2],
    with_noise="bv",
)
fig.set_size_inches(10, 30)
plt.show()
# color by terminal state label

### Appendix

With larger values of N, less likely to see such a unanimously effcting sequence of perturbations, but I'm not really sure that's the analogy anyway.

### Conceptual background, introduction:

#### Perturbed Boolean networks:

Let Boolean network functions, input assignments be immutable; Each sequence of unit perturbations is fired probabilistically (a la Petri-net);

Consider a Gene Regulatory Network whose edges are derived from correlations of differentially expressed genes, from measurements of abundances of protein, RNA over a panel of perturbations or natural variation.

How different would such GRNs really be for unperturbed, and perturbed networks? In terms of design-space of perturbations / interventions, the result of weighing edges is likely not sufficiently unique to specify a cassette, further, it would not (TBD) pose a subset of subproblems (???)

As part of backtracking, a condition could be applied to the as-would-be-inferred GRN, such that ??? maybe we can expect that healthy cellular systems, with such an altered GRN, woudl still behave healthily.

#### Problem specification:

Find a sequence of unit perturbations (p1, p2) such that for all cycles (except goal cycle) there exists an interval L such that a run-in from the perturbed state (i.e. p1 has been applied, L steps have occurred, and p2 has been applied) would terminate in cycle detection of the goal cycle.

#### Assumptions:

Perturbation does not fire in goal states. (This can be relaxed, but I'm not sure it would work, practically, this could be embodied by a constitutive or conditional payload in the perturbation that fires only in the goal state, causing cell death)

#### Implication:

If such a sequential perturbation exists, then, where it is applied probabilistically to a distribution of network states, eventually, all trajectories will be in goal states.

---
Applications of a similar search, for scaled and meaningfully assigned Boolean networks, includes designed changes to cancer ecosystems.

In [ ]:
# # there is a topo sort to be applied here ...
# # for row in (all_matrix @ all_matrix)[:-1]:
# #     print('\t'.join(str(s) for s in row[:-1]))
# shuffle(matrices)
# prod_sum = np.eye(N=len(matrices[0]), M=len(matrices[0]))
# for m in matrices:
#     prod_sum = prod_sum @ m
# # for i in range(len(prod_sum)):
# #     for j in range(len(prod_sum)):
# #         if prod_sum[i][j] > 0:
# #             prod_sum[i][j] = 1
# plt.imshow(prod_sum)  # ah- here is a way to cluster the cycles  # this one is potentially useful, at least as represented, can section into partitions# for output TODO pickle
print(net)
for n in net.nodes:
    print(n.function)
print(net.node_inputs_assignments)
# BooleanNetwork: N=40. avg_k= 1.65. Run-ins: 1202. Observed cycles: 30. t_records: 1202. u_records: 0  # two clear sections of numbers
# Cycle lengths: [2, 2, 3, 3, 3, 3, 4, 4, 24, 2, 12, 24, 12, 2, 2, 2, 6, 2, 2, 6, 6, 1, 6, 1, 8, 4, 4, 2, 2, 2]
# <package.boolean_networks._BooleanFunction6 object at 0x127b6bda0>
# <package.boolean_networks._BooleanFunction15 object at 0x127b6bf50>
# <package.boolean_networks._BooleanFunction15 object at 0x127b6bf50>
# <package.boolean_networks._BooleanFunction5 object at 0x127b6bd70>
# <package.boolean_networks._BooleanFunction3 object at 0x127b6bd10>
# <package.boolean_networks._BooleanFunction9 object at 0x127b6be30>
# <package.boolean_networks._BooleanFunction2 object at 0x127b6bce0>
# <package.boolean_networks._BooleanFunction12 object at 0x127b6bec0>
# <package.boolean_networks._BooleanFunction13 object at 0x127b6bef0>
# <package.boolean_networks._BooleanFunction5 object at 0x127b6bd70>
# <package.boolean_networks._BooleanFunction6 object at 0x127b6bda0>
# <package.boolean_networks._BooleanFunction8 object at 0x127b6be00>
# <package.boolean_networks._BooleanFunction15 object at 0x127b6bf50>
# <package.boolean_networks._BooleanFunction9 object at 0x127b6be30>
# <package.boolean_networks._BooleanFunction5 object at 0x127b6bd70>
# <package.boolean_networks._BooleanFunction13 object at 0x127b6bef0>
# <package.boolean_networks._BooleanFunction11 object at 0x127b6be90>
# <package.boolean_networks._BooleanFunction12 object at 0x127b6bec0>
# <package.boolean_networks._BooleanFunction13 object at 0x127b6bef0>
# <package.boolean_networks._BooleanFunction15 object at 0x127b6bf50>
# <package.boolean_networks._BooleanFunction6 object at 0x127b6bda0>
# <package.boolean_networks._BooleanFunction3 object at 0x127b6bd10>
# <package.boolean_networks._BooleanFunction15 object at 0x127b6bf50>
# ...
# <package.boolean_networks._BooleanFunction10 object at 0x127b6be60>
# <package.boolean_networks._BooleanFunction9 object at 0x127b6be30>
# <package.boolean_networks._BooleanFunction2 object at 0x127b6bce0>
# [[3, 0], [20, 34], [7, 16], [21, 9], [20, 33], [0, 4], [9, 17], [20, 32], [14, 19], [9, 26], [34, 7], [37, 5], [26, 24], [39, 4], [2, 16], [16, 30], [12, 39], [5, 34], [13, 13], [3, 27], [28, 14], [23, 23], [7, 37], [24, 24], [15, 22], [17, 31], [31, 8], [22, 39], [6, 10], [25, 9], [33, 23], [33, 3], [7, 10], [12, 37], [25, 11], [38, 14], [36, 36], [10, 12], [36, 31], [18, 5]]
# Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...